Join parking spot points to other shapefiles to get counts of parking spots in various geographies.

Save out to a new geojson with additional fields.

In [1]:
import pandas as pd
import geopandas as gpd

## Spatial Join saga

geopandas says I need pygeos or rtreet to do a spatial join. https://geopandas.org/install.html

### pip install pygeos

This breaks the gpd.read_file() for the ANC shapefile. Get the error `Shell is not a LinearRing Error`. Uninstalling pygeos

### pip install rtree

`ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.`

Trying `brew install spatialindex`

based off of https://stackoverflow.com/questions/59981914/missing-dependancies-of-rtree

`pip install rtree` works

In [2]:
estimated_spots = gpd.read_file('output/estimated_rpp_spots.geojson')

## ANC

In [3]:
anc = gpd.read_file(
    'input/Advisory_Neighborhood_Commissions_from_2013-shp/Advisory_Neighborhood_Commissions_from_2013.shp')

In [4]:
anc_spots = gpd.sjoin(estimated_spots, anc[['geometry', 'ANC_ID']], how='left', op='intersects')

In [5]:
anc_spots.drop('index_right', axis=1, inplace=True)

In [6]:
anc_spots.head()

,parking_spot_id,source_street_objectid,street_name,street_type,quadrant,block_number,ward,blkstreet,block_side,geometry,ANC_ID
0,129004,1972000,MACARTHUR,BLVD,NW,5800.0,3,5800 MACARTHUR BLVD NW,both sides,POINT (-77.11153 38.93556),3D
1,129003,1972000,MACARTHUR,BLVD,NW,5800.0,3,5800 MACARTHUR BLVD NW,both sides,POINT (-77.11149 38.93551),3D
2,128980,1972000,MACARTHUR,BLVD,NW,5800.0,3,5800 MACARTHUR BLVD NW,both sides,POINT (-77.11148 38.93559),3D
3,129002,1972000,MACARTHUR,BLVD,NW,5800.0,3,5800 MACARTHUR BLVD NW,both sides,POINT (-77.11146 38.93546),3D
4,128979,1972000,MACARTHUR,BLVD,NW,5800.0,3,5800 MACARTHUR BLVD NW,both sides,POINT (-77.11144 38.93554),3D


## Census Block

In [7]:
cb = gpd.read_file('input/tl_2019_11_tabblock10/tl_2019_11_tabblock10.shp')

In [8]:
output_crs = ('epsg', '4326')
cb = cb.to_crs(output_crs)

In [9]:
# GEOID10 seems to be the field with tract, block group, and block
anc_cb_spots = gpd.sjoin(anc_spots, cb[['geometry', 'GEOID10']], how='left', op='intersects')

In [10]:
anc_cb_spots.drop('index_right', axis=1, inplace=True)

## Census Tract

In [11]:
anc_cb_spots['census_tract'] = anc_cb_spots['GEOID10'].str[5:11]

## Save Output

In [12]:
def geojson_to_csv(input_geojson):
    """
    Save a copy of a GeoJSON file as a CSV
    """

    gdf = gpd.read_file(input_geojson)
    
    gdf['longitude'] = gdf['geometry'].x
    gdf['latitude'] = gdf['geometry'].y

    columns_to_csv = [c for c in gdf if c != 'geometry']
    output_filename = input_geojson.replace('.geojson', '.csv')
    gdf.to_csv(output_filename, index=False)
    print(f'CSV saved to: {output_filename}')


In [13]:
anc_cb_spots.head()

,parking_spot_id,source_street_objectid,street_name,street_type,quadrant,block_number,ward,blkstreet,block_side,geometry,ANC_ID,GEOID10,census_tract
0,129004,1972000,MACARTHUR,BLVD,NW,5800.0,3,5800 MACARTHUR BLVD NW,both sides,POINT (-77.11153 38.93556),3D,110010009022005,000902
1,129003,1972000,MACARTHUR,BLVD,NW,5800.0,3,5800 MACARTHUR BLVD NW,both sides,POINT (-77.11149 38.93551),3D,110010009022005,000902
2,128980,1972000,MACARTHUR,BLVD,NW,5800.0,3,5800 MACARTHUR BLVD NW,both sides,POINT (-77.11148 38.93559),3D,110010009011003,000901
3,129002,1972000,MACARTHUR,BLVD,NW,5800.0,3,5800 MACARTHUR BLVD NW,both sides,POINT (-77.11146 38.93546),3D,110010009022005,000902
4,128979,1972000,MACARTHUR,BLVD,NW,5800.0,3,5800 MACARTHUR BLVD NW,both sides,POINT (-77.11144 38.93554),3D,110010009011003,000901


In [14]:
output_file = 'output/estimated_rpp_spots_anc_census.geojson'
anc_cb_spots.to_file(output_file, driver='GeoJSON')

In [15]:
geojson_to_csv(output_file)

CSV saved to: output/estimated_rpp_spots_anc_census.csv
